**Tutorial 6 - Regression and parameter estimation on Type Ia Supernovae data**

In this tutorial we will experiment with some type Ia supernova data.  This is part of the data used to determine that Universe's expansion is accelerating.

Data obtained is from the Supernova Cosmology Project at:
http://supernova.lbl.gov/union/descriptions.html#Magvsz

Background:

The observed magnitude of an object with luminosity $L$ is

$m = - 2.5 \log\left( \frac{L}{2\pi D_L^2} \right) + m_o = 5 \log\left( D_L \right) + 2.5 \log\left( L \right) + m_o$

where $D_L$ is the luminosity distance.  The peak luminosity of a type Ia supernovae 
is directly related to the width of its lightcurve and its color.  In this data set, the correction to a standard candle has already been done (we will look at this in a later tutorial) and it is reported in terms of the estimated distance modulus

$\mu = 5 \log\left( D_L \right) - 5$

This assumes a Hubble constant ($D_L \propto 1/H_o$) and requires a calibration using other distance indicators in local galaxies so there is an additive constant to the distance modulus or a multiplicative constant to the brightness that is not very well constrained, ie the relative magnatudes of the supernovae are well measured but not their absolute brightnesses.

General relativity and the energy content of the Universe predicts a function for $D_L(z)$ (or $\mu(z)$) where $z$ is the cosmological redshift of the supernovae.

In this tutorial we will make and test some linear models for $\mu(z)$.  It will be assumed that the measured redshifts are very well measured so that thier erros do not need to be taken into account.  (We will also ignore pecular velocities.) We will also see if a flat Universe without a cosmological constant is an adequate fit to the data under some approximations.

1) Import the data using the command. 

`data = pa.read_csv("SCPUnion2.1_mu_vs_z.txt",sep='\t',comment='#')`

Note that there are comments starting with 
"#" and the seporators are tabs not commas.

Plot the distance modulus vs redshift with error bars.  Label the axes.

2) Try fitting these the $\mu(z)$ data with a polynomial models of order 1,2 and 3 in $z$ using numpy.polyfit(). Use the w parameter in this function to include the errors from the data files in these $\chi^2$ fits.  Plot these models on top of the data.  Leave the errors out to improve clarity.  Do these make good models?

3) Let's try fitting a model to the luminosity distance instead of the distance modulus.  

Convert the distance modulus into luminosity distance
$D_L = A ~10^{\mu/5}$

were we don't know the constant $A$ because of calibration and uncertainties in the Hubble constant.

You can estimate the errors in the distance using the expansion

$ \delta D_L \sim \frac{\ln(10)}{5} ~ D_L \delta \mu$

,the first order expansion of $D_L$ in terms of $\mu$.  This is only an approximation.

We want to fit a second order polynomial to $D_L$ vs $z$ with a zero intercept, i.e. the constant term is zero.  This is because we know that $D_L(z=0)=0$.

numpy.polyfit() does not allow for a zero intercept.  So we will use numpy.linalg.lstsq(M,y) to do the fit.  First you need to construct the M matrix which is of the form
\begin{equation}
M = \left(
\begin{array}{cc}
z_1^2 & z_1 \\
z_2^2 & z_2 \\
\vdots & \vdots \\
z_i^2 & z_i
\end{array}
\right)
\end{equation}
The $y$'s in this case are the $D_L$'s.  Now we need to weight by the errors.  There doesn't seem to be a function for doing this automatically in python, but we can do it ourselves.  If the dependent variables and the columns of the M matrix are divided by the standard deviations this will be equivalent to the chi-squared weighting as described in the lecture notes.

Fit a second order polynomial to $D_L$ vs $z$. Plot it and the data.

In [1]:

## weighted least squares
def weighted_lstsq(M,y,sigma):
    yw = ...
    Mw = ...
    return np.linalg.lstsq(Mw,yw,rcond=None)


4) General relativity and the standard cosmological model predict that to 
second order

$D_L(z) \simeq \frac{c}{H_o} \left[ z + \frac{z^2}{2}(1-q_o) \right]$

where $q_o$ is called the deceleration parameter which is given by

$q_o = \frac{\Omega_m}{2} - \Omega_\Lambda$

where $\Omega_m$ is the density of the Universe in ordinary matter in units of the critical density and $\Omega_\Lambda$ is the energy density of the cosmological constant in the same units.

If the Universe is geometrically flat $\Omega_m+ \Omega_\Lambda = 1$ and 

$q_o = \frac{3}{2}\Omega_m - 1$

If there is no cosmological constant (the expansion is not accelerating) $q_o=1/2$.

We have found a second order model already.  You can see that $q_o$ is a combination of linear model coefficients. You can also see that $p[0]/p[1] = (1-q_o)/2$ where $p[]$ are the model coefficients.

For a flat universe with no cosmological constant $\Omega_m = 1$, $\Omega_\Lambda =0$  $p[0]/p[1] = 1/4$.  

What is your measured value for $q_o$?  If the universe is flat what is your measured value for $\Omega_m$?

5) This looks good, but the errors are not really on $D_L$.  They are on 
the distance modulus.  And we don't have an error on $q_o$ because it is a nonlinear combination of linear parameters.

What we really want to find is a nonlinear model.  For this let's use scipy.optimize.curve_fit().  For this we must 

a) Define a function func_modulus($z,\mu_o,q_o$) which returns 

$\mu(z) = 5\log_{10}\left[ z + \frac{1}{2}(1-qo)z^2\right] + \mu_o$


In [ ]:
def func_modulus(z,muo,qo):
    ...

b) Use scipy.optimize.curve_fit() to find the best fit model.  Use the sigma and absolute_sigma=True options to include the errors.

The outputs of this function are model,covariance.

What is the $q_o$ you get in this way?

c) Use func_modulus(z,muo,qo) to plot the best fit model over the data.

5) Calculate the $\chi^2$ for this model and data.  Remember that 

$\chi^2 = \sum_i \frac{ [ y_i - model(x_i)~ ]^2}{\sigma^2} $

where $model(x_i)$ is the best fit model.

Is this an acceptable fit?  Use the $\chi^2$ distribution to find out.  Use N-2 degrees of freedom. (hint: Use scipy.stats.chi2.cdf() to find its significance. )

6) Why might a person object to using a $\chi^2_{n-2}$ distribution 
above in determining the significance of the result?

7) Plot the $\chi^2$ p-value with $n$ degrees of freedom as a function of $q_o$ over a range that includes the best fit value and extends to where it is a small fraction of its peak.

7) Let's try finding error bars on our measurement of $q_o$ in a different way.

Find the bootstrap variance of your estimate of $q_o$.  This will require you to make a new data sets by sampling from the original one with replacement (Hint: numpy.random.choice(N,N) will give you a random set of integers between 0 and N-1).  Then find $q_o$ from this data set.  Put this in a loop and repeat 1000 times recording the $q_o$ value each time in a vector of $q_o$'s. 

Make a nice histogram of the $q_o$ vector.

This vector of $q_o$'s is approximately sampled from the true distribution.  Find the mean and variance of this vector.  This is an estimate of the variance in the estimate of $q_o$.

What is the measured value of $q_o$ with errors?

Assuming the Universe is flat what is the measured value of $\Omega_m$ with errors?

In [5]:
### put loop here


In [4]:
### put plot here
.
.

print('qo = ', ... ,' +/- ', ... )
print('Omega_m = ', ... ,' +/- ', ... )


Comment:  In this tutorial we have made several approximations that might not be justified.  One is that we use an expansion for $D_L$ as a function of $z$ that is not really valid at z ~ 1.  If we use the correct relationship and calculate the errors in a better way we might get a different answer for $\Omega_m$.  Also we have assumed the Universe geometricly flat.  We will do better in a future tutorial!